In [7]:
!pip install gradio SpeechRecognition pydub indic-transliteration

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 50.4/50.4 kB 4.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 12.5/12.5 MB 88.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 318.6/318.6 kB 30.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 75.6/75.6 kB 8.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 77.9/77.9 kB 8.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 141.1/141.1 kB 16.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 10.2/10.2 MB 82.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 62.8/62.8 kB 6.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 93.1/93.1 kB 10.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 58.3/58.3 kB 6.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 71.9/71.9 kB 7.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 130.2/130.2 kB 12.3 MB/s eta 0:00:00
  Attempting uninstall: tomlkit
    Found existing

In [10]:
import gradio as gr
import speech_recognition as sr
from pydub import AudioSegment
from pydub.utils import mediainfo
from indic_transliteration import sanscript
from indic_transliteration.sanscript import transliterate

def convert_audio_format(uploaded_file):
    audio = AudioSegment.from_file(uploaded_file)
    converted_file = "converted_audio.wav"
    audio.export(converted_file, format="wav")
    return converted_file

def transcribe_audio(audio_file, language_code):
    recognizer = sr.Recognizer()
    with sr.AudioFile(audio_file) as source:
        audio_data = recognizer.record(source)
        transcribed_text = recognizer.recognize_google(audio_data, language=language_code)
    return transcribed_text

def get_word_timestamps(transliterated_text, search_word, audio_duration):
    words = transliterated_text.split()
    total_words = len(words)
    word_duration = audio_duration / total_words
    occurrences = []
    for i, word in enumerate(words):
        if search_word.lower() in word.lower():
            start_time = i * word_duration
            end_time = start_time + word_duration
            occurrences.append((start_time, end_time))
    return occurrences

def process_audio(audio_file, language_code, search_word):
    converted_file = convert_audio_format(audio_file)
    transcribed_text = transcribe_audio(converted_file, language_code)

    if language_code == "te":
        transliterated_text = transliterate(transcribed_text, sanscript.TELUGU, sanscript.ITRANS)
    elif language_code == "hi":
        transliterated_text = transliterate(transcribed_text, sanscript.DEVANAGARI, sanscript.ITRANS)
    elif language_code == "ta":
        transliterated_text = transliterate(transcribed_text, sanscript.TAMIL, sanscript.ITRANS)
    elif language_code == "ml":
        transliterated_text = transliterate(transcribed_text, sanscript.MALAYALAM, sanscript.ITRANS)
    elif language_code == "kn":
        transliterated_text = transliterate(transcribed_text, sanscript.KANNADA, sanscript.ITRANS)
    else:
        transliterated_text = transcribed_text

    audio_info = mediainfo(converted_file)
    audio_duration = float(audio_info['duration'])

    timestamps = get_word_timestamps(transliterated_text, search_word, audio_duration)

    return transcribed_text, transliterated_text, timestamps

iface = gr.Interface(
    fn=process_audio,
    inputs=[
        gr.Audio(type="filepath", label="Upload Audio File"),
        gr.Radio(["en", "te", "hi", "ta", "ml", "kn"], label="Select Language Code"),
        gr.Textbox(label="Enter the word to search for timestamps")
    ],
    outputs=[
        gr.Textbox(label="Transcribed Text"),
        gr.Textbox(label="Transliterated Text (English)"),
        gr.Textbox(label="Timestamps")
    ],
    title="Audio Transcription and Word Timestamp Extraction"
)

iface.launch()

Setting queue=True in a Colab notebook requires sharing enabled. Setting `share=True` (you can turn this off by setting `share=False` in `launch()` explicitly).

Colab notebook detected. To show errors in colab notebook, set debug=True in launch()
Running on public URL: https://f2898936a863b24e63.gradio.live

This share link expires in 72 hours. For free permanent hosting and GPU upgrades, run `gradio deploy` from Terminal to deploy to Spaces (https://huggingface.co/spaces)
